In [ ]:
!pip install yolo torch ultralytics torchvision 
!gdown 10JUYDFfmnm3AUmYHieYIIel-tJkU-66y

In [ ]:
!yolo task=detect mode=predict model=/content/best.pt \
    conf=0.5 source=/content/1 \
    save_txt=True agnostic_nms=True line_width=0 save=True

In [ ]:
import os
import cv2
from pathlib import Path

# Define paths
results_folder = Path('/content/runs/detect/predict')
labels_folder = results_folder / 'labels'
output_folder = Path('/content/detected_objects')

# Create the output folder if it doesn't exist
output_folder.mkdir(parents=True, exist_ok=True)

# Loop through each label file in the labels directory
for label_file in labels_folder.glob('*.txt'):
    # Get the base name of the file (e.g., 'check' from 'check.txt')
    image_name = label_file.stem
    image_extensions = ['.jpg', '.jpeg', '.png']  # Add any other relevant extensions
    image_file = None
    
    # Find the corresponding image file (searching for multiple possible extensions)
    for ext in image_extensions:
        possible_image_file = results_folder / f'{image_name}{ext}'
        if possible_image_file.exists():
            image_file = possible_image_file
            break
    
    if not image_file:
        print(f"Image file corresponding to {label_file} not found.")
        continue

    # Load the image using OpenCV
    img = cv2.imread(str(image_file))
    if img is None:
        print(f"Failed to load image: {image_file}")
        continue

    # Get image dimensions
    height, width, _ = img.shape

    # Read the detection results from the .txt file
    with open(label_file, 'r') as file:
        detections = file.readlines()

    # Loop through each detection in the label file
    for i, detection in enumerate(detections):
        parts = detection.strip().split()
        class_id = parts[0]  # The class ID is the first element in the line
        
        # Handle cases with 4 or 5 elements in the line
        if len(parts) == 5:
            x_center, y_center, bbox_width, bbox_height = map(float, parts[1:])
        elif len(parts) == 6:
            x_center, y_center, bbox_width, bbox_height, _ = map(float, parts[1:])

        # Convert normalized coordinates back to image size
        x_center = int(x_center * width)
        y_center = int(y_center * height)
        bbox_width = int(bbox_width * width)
        bbox_height = int(bbox_height * height)

        # Calculate the top-left and bottom-right corners of the bounding box
        x_min = int(x_center - bbox_width / 2)
        y_min = int(y_center - bbox_height / 2)
        x_max = int(x_center + bbox_width / 2)
        y_max = int(y_center + bbox_height / 2)

        # Ensure coordinates are within image bounds
        x_min = max(0, x_min)
        y_min = max(0, y_min)
        x_max = min(width, x_max)
        y_max = min(height, y_max)

        # Crop the object from the image
        cropped_img = img[y_min:y_max, x_min:x_max]

        # Create a directory for this class if it doesn't exist
        class_folder = output_folder / f'class_{class_id}'
        class_folder.mkdir(parents=True, exist_ok=True)

        # Save the cropped image in the class folder
        destination_file = class_folder / f'{image_file.stem}_{i}{image_file.suffix}'
        cv2.imwrite(str(destination_file), cropped_img)

print("Objects have been cropped and saved in class-specific folders.")
